In [1]:
import os
import sys
import pandas as pd
import datetime
import numpy as np
import time
import re
from tqdm import tqdm
import timeit
tqdm.pandas()
import omap


In [ ]:
starttime = timeit.default_timer()

def test_folder_path (input_folder_path, start_time, end_time):
    # Retrieve Train number
    train_number = re.findall(r'%s(\d+)' %'Train ', input_folder_path)[0]
    
    # Determine time difference between start and end time
    datetime_format = '%Y/%m/%d %H:%M:%S'
    start_time = datetime.datetime.strptime(start_time, datetime_format)
    end_time = datetime.datetime.strptime(end_time, datetime_format)
    diff = end_time - start_time
    hours = diff.total_seconds() /3600

    start_time_new = start_time.strftime('%m/%d/%Y %H:%M:%S')
    end_time_new = end_time.strftime('%m/%d/%Y %H:%M:%S')
    result_list = []
    for car in os.listdir(input_folder_path):
        car_number = re.findall(r'%s(\d+)' %'Car ', car)[0]
        print("---Processing Train " + train_number + " Car " + car_number + "---")
        date = os.listdir(os.path.join(input_folder_path, car))[0]
        flag = True
        back_date_flag = False
        # Test for correct folder structure
        try:
            datetime.datetime.strptime(date, "%y%m%d")
            #print("Correct date string format.")
        except ValueError:
            print("Inorrect date string format. Example 200116 %y%m%d. This might result in process error.")
            flag = False    
        
        # OMAP Processing
        
        # Look for ATO, ATP, COM and TDMS folder
        if not os.path.exists(os.path.join(input_folder_path, car, date, 'OMAP_ATO')):
            print("ATO folder does not exist. Incorrect folder structure")
            flag = False
        else:
            ATO_file_list = os.listdir(os.path.join(input_folder_path, car, date, 'OMAP_ATO'))       
            sorted(ATO_file_list)
            # Find dataframe/append for the given start and end time
            ATO_dataframe = pd.DataFrame()
            ATP_dataframe = pd.DataFrame()
            COM_dataframe = pd.DataFrame()
            TDMS_dataframe = pd.DataFrame()
            for index in range (0, len(ATO_file_list)):
            #for file in ATO_file_list:
                file = ATO_file_list[index]
                process_flag = True
                # For each ATO file, finding corresponding ATP, COM and TDMS file
                ATP_file_list = os.listdir(os.path.join(input_folder_path, car, date, 'OMAP_ATP'))
                if not file.replace('ATO','ATP') in ATP_file_list:
                    print(file + " does not have a corresponding ATP file. Hence will not be processed. ")
                    process_flag = False
                
                COM_file_list = os.listdir(os.path.join(input_folder_path, car, date, 'OMAP_COM'))
                if not file.replace('ATO','COM') in COM_file_list:
                    print(file + " does not have a corresponding COM file. Hence will not be processed. ")
                    process_flag = False
                    
                TDMS_file_list = os.listdir(os.path.join(input_folder_path, car, date, 'OMAP_TDMS'))
                if not file.replace('ATO','TDMS') in TDMS_file_list:
                    print(file + " does not have a corresponding TDMS file. Hence will not be processed. ")
                    process_flag = False               
                
                if process_flag:
                    # Read ATO log file and find relevant rows based on start and end time)
                    df_ATO = pd.read_csv(os.path.join(input_folder_path, car, date, 'OMAP_ATO', file), sep="\t")
                    df_ATO['Date'] = pd.to_datetime(df_ATO['Date'], format='%m/%d/%Y %H:%M:%S')
                    # Read ATP log file and find relevant rows based on start and end time)
                    df_ATP = pd.read_csv(os.path.join(input_folder_path, car, date, 'OMAP_ATP', file.replace('ATO','ATP')), sep="\t")
                    df_ATP['Date'] = pd.to_datetime(df_ATP['Date'], format='%m/%d/%Y %H:%M:%S')
                    # Read COM log file and find relevant rows based on start and end time)
                    df_COM = pd.read_csv(os.path.join(input_folder_path, car, date, 'OMAP_COM', file.replace('ATO','COM')), sep="\t")
                    df_COM['Date'] = pd.to_datetime(df_COM['Date'], format='%m/%d/%Y %H:%M:%S')
                    # Read TDMS log file and find relevant rows based on start and end time)
                    df_TDMS = pd.read_csv(os.path.join(input_folder_path, car, date, 'OMAP_TDMS', file.replace('ATO','TDMS')), sep="\t")
                    df_TDMS['Date'] = pd.to_datetime(df_TDMS['Date'], format='%m/%d/%Y %H:%M:%S')
                                        
                    if hours == 1: # Process only 1 log file
                        # Compare between cuurent index and next index (to be removed as it doesnt max sense to do it this way)
                        # For the sake of being the same as ATLAS
                        if (index+1 < len(ATO_file_list)):
                            df_ATO_next = pd.read_csv(os.path.join(input_folder_path, car, date, 'OMAP_ATO', ATO_file_list[index+1]), sep="\t")
                            df_ATO_next['Date'] = pd.to_datetime(df_ATO_next['Date'], format='%m/%d/%Y %H:%M:%S')
                            next_length = len((df_ATO_next.loc[(df_ATO_next['Date'] == start_time_new)]))
                            current_length = len((df_ATO.loc[(df_ATO['Date'] == start_time_new)]))
                            
                            if next_length > current_length:
                                # Read logs from next index
                                df_ATP_next = pd.read_csv(os.path.join(input_folder_path, car, date, 'OMAP_ATP', ATO_file_list[index+1].replace('ATO','ATP')), sep="\t")
                                df_ATP_next['Date'] = pd.to_datetime(df_ATP_next['Date'], format='%m/%d/%Y %H:%M:%S')
                                df_COM_next= pd.read_csv(os.path.join(input_folder_path, car, date, 'OMAP_COM', ATO_file_list[index+1].replace('ATO','COM')), sep="\t")
                                df_COM_next['Date'] = pd.to_datetime(df_COM_next['Date'], format='%m/%d/%Y %H:%M:%S')
                                df_TDMS_next = pd.read_csv(os.path.join(input_folder_path, car, date, 'OMAP_TDMS', ATO_file_list[index+1].replace('ATO','TDMS')), sep="\t")
                                df_TDMS_next['Date'] = pd.to_datetime(df_TDMS_next['Date'], format='%m/%d/%Y %H:%M:%S') 
                                
                                ATO_dataframe = ATO_dataframe.append(df_ATO_next.loc[(df_ATO_next['Date'] >= start_time_new) & (df_ATO_next['Date'] <= end_time_new)])
                                ATP_dataframe = ATP_dataframe.append(df_ATP_next.loc[(df_ATP_next['Date'] >= start_time_new) & (df_ATP_next['Date'] <= end_time_new)])
                                COM_dataframe = COM_dataframe.append(df_COM_next.loc[(df_COM_next['Date'] >= start_time_new) & (df_COM_next['Date'] <= end_time_new)])
                                TDMS_dataframe = TDMS_dataframe.append(df_TDMS_next.loc[(df_TDMS_next['Date'] >= start_time_new) & (df_TDMS_next['Date'] <= end_time_new)])
                                break
                            else:
                                # Read logs from current index
                                ATO_dataframe = ATO_dataframe.append(df_ATO.loc[(df_ATO['Date'] >= start_time_new) & (df_ATO['Date'] <= end_time_new)])
                                ATP_dataframe = ATP_dataframe.append(df_ATP.loc[(df_ATP['Date'] >= start_time_new) & (df_ATP['Date'] <= end_time_new)])
                                COM_dataframe = COM_dataframe.append(df_COM.loc[(df_COM['Date'] >= start_time_new) & (df_COM['Date'] <= end_time_new)])
                                TDMS_dataframe = TDMS_dataframe.append(df_TDMS.loc[(df_TDMS['Date'] >= start_time_new) & (df_TDMS['Date'] <= end_time_new)])
                                break
                        else:
                            # Read logs from only index
                            ATO_dataframe = ATO_dataframe.append(df_ATO.loc[(df_ATO['Date'] >= start_time_new) & (df_ATO['Date'] <= end_time_new)])
                            ATP_dataframe = ATP_dataframe.append(df_ATP.loc[(df_ATP['Date'] >= start_time_new) & (df_ATP['Date'] <= end_time_new)])
                            COM_dataframe = COM_dataframe.append(df_COM.loc[(df_COM['Date'] >= start_time_new) & (df_COM['Date'] <= end_time_new)])
                            TDMS_dataframe = TDMS_dataframe.append(df_TDMS.loc[(df_TDMS['Date'] >= start_time_new) & (df_TDMS['Date'] <= end_time_new)])
                            break
                    else:
                        # Need to fix for whole number hours (probably not because dont have to copy how atlas process which files)
                        ATO_dataframe = ATO_dataframe.append(df_ATO.loc[(df_ATO['Date'] >= start_time_new) & (df_ATO['Date'] < end_time_new)])
                        ATP_dataframe = ATP_dataframe.append(df_ATP.loc[(df_ATP['Date'] >= start_time_new) & (df_ATP['Date'] < end_time_new)])
                        COM_dataframe = COM_dataframe.append(df_COM.loc[(df_COM['Date'] >= start_time_new) & (df_COM['Date'] < end_time_new)])
                        TDMS_dataframe = TDMS_dataframe.append(df_TDMS.loc[(df_TDMS['Date'] >= start_time_new) & (df_TDMS['Date'] < end_time_new)])
                        # Check if log is '00'_00_00 time and within time period
                        if re.findall(r'%s(\d+)' %'_', file)[0] == '00' and len(df_ATO.loc[(df_ATO['Date'] >= start_time_new) & (df_ATO['Date'] < end_time_new)])>0:
                            back_date_flag = True
                else:
                    sys.exit(car + " data folder is incorrect / missing corresponding log files. Cannot proceed with processing! Kindly check whether ATO, ATP COM and TDMS have their corresponding log files.")
            
            
            # Check if log is '00'_00_00 time and within time period
            if re.findall(r'%s(\d+)' %'_', file)[0] == '00' and len(ATO_dataframe.loc[(ATO_dataframe['Date'] >= start_time_new) & (ATO_dataframe['Date'] <= end_time_new)])>0:
                back_date_flag = True
                print("back_date_flag")
                        
            ATO_dataframe = ATO_dataframe.reset_index(drop=True)
            ATP_dataframe = ATP_dataframe.reset_index(drop=True)
            COM_dataframe = COM_dataframe.reset_index(drop=True)
            TDMS_dataframe = TDMS_dataframe.reset_index(drop=True)
            
            if(ATO_dataframe.shape[0] == 0):
                sys.exit("No matching time window from start to end time! Kindly check if you have input the correct start and end time!")
            
            #print(ATO_dataframe.tail(30))
            # Process ATO, ATP, COM, TDMS individually
            ATO_dataframe = omap.process_ato(ATO_dataframe, back_date_flag)
            ATP_dataframe = omap.process_others(ATP_dataframe, 'ATP')
            COM_dataframe = omap.process_others(COM_dataframe, 'COM')
            TDMS_dataframe = omap.process_others(TDMS_dataframe, 'TDMS')

            # Merge ATO-ATP, ATO-COM, ATO-TDMS
            ATO_ATP_result = omap.merge_ato_n_others(ATO_dataframe, ATP_dataframe)
            ATO_COM_result = omap.merge_ato_n_others(ATO_dataframe, COM_dataframe)
            ATO_TDMS_result = omap.merge_ato_n_others(ATO_dataframe, TDMS_dataframe)

            # Merge all results dataframes
            df_result = omap.merge_all(ATO_ATP_result, ATO_COM_result, ATO_TDMS_result, train_number, car_number, start_time, end_time)
            
            # Append results for Unit Test
            result_list.append([ATO_ATP_result, ATO_COM_result, ATO_TDMS_result, df_result])
            
    return result_list
                          
            
# input_folder_path =  './T20 OMAP DATA/Train 20 CSV/'
# #input_folder_path =  './test_folder/Train 19 CSV/'
# #Machine Time of interest ('%Y/%m/%d %H:%M:%S')
# start_time = '2020/01/16 05:00:00' 
# end_time = '2020/01/16 08:00:00'

# Machine Time of interest ('%m/%d/%Y %H:%M:%S')
# start_time = '01/16/2020 07:00:00' 
# end_time = '01/16/2020 08:00:00'
#sample_output_filename = ['./OMAP_Train_20_Car_39_20200116_0600_to_20200116_0700.csv']
#sample_output_filename = ['./OMAP_Train_20_Car_39_20200116_0700_to_20200116_0800.csv']
#sample_output_filename = ['./OMAP_Train_20_Car_39_20200116_0500_to_20200116_0800.csv']

input_folder_path = './Train 23 14NOV2019 Work Folder/raw logs/OMAP/Train 23 Copy'
start_time = '2019/11/14 17:00:00' 
end_time = '2019/11/14 20:00:00'
# sample_output_filename = ['./Train 23 14NOV2019 Work Folder/preprocessing_output/OMAP_Train_23_Car_45_20191114_1700_to_20191114_1800.csv',
#                           './Train 23 14NOV2019 Work Folder/preprocessing_output/OMAP_Train_23_Car_46_20191114_1700_to_20191114_1800.csv']
# sample_output_filename = ['./Train 23 14NOV2019 Work Folder/preprocessing_output/OMAP_Train_23_Car_46_20191114_1800_to_20191114_1900.csv']
sample_output_filename = ['./Train 23 14NOV2019 Work Folder/preprocessing_output/OMAP_Train_23_Car_45_20191114_1700_to_20191114_2000.csv']


# input_folder_path = './Train 10 26FEB2018 Work Folder/raw_logs/OMAP/Train 10'
# start_time = '2018/02/26 01:00:00' 
# end_time = '2018/02/26 02:00:00' 
# sample_output_filename = ['./Train 10 26FEB2018 Work Folder/preprocessed_output/OMAP_Train_10_Car_19_20180226_0100_to_20180226_0200.csv']

# input_folder_path = './Train 10 22FEB2018 Work Folder/raw_logs/OMAP/Train 10'
# start_time = '2018/02/22 01:00:00' 
# end_time = '2018/02/22 02:00:00' 
# sample_output_filename = ['./Train 10 22FEB2018 Work Folder/preprocessed_output/OMAP_Train_10_Car_19_20180222_0100_to_20180222_0200.csv']


result_list = test_folder_path(input_folder_path, start_time, end_time)
print("Time Taken:", timeit.default_timer() - starttime)

for index in range (0, len(result_list)):
    # Unit Test
    omap.unit_test(sample_output_filename[index], 'ATP', result_list[index][0])
    omap.unit_test(sample_output_filename[index], 'COM', result_list[index][1])
    omap.unit_test(sample_output_filename[index], 'TDMS', result_list[index][2])
    omap.unit_test_all(sample_output_filename[index], result_list[index][3])


---Processing Train 23 Car 45---
---Processing ATO Dataframe---
Number of null value entry =  0
Found non-monotonic sequence at index:  Int64Index([25122, 41971, 51783, 57456, 85700, 89530, 95312], dtype='int64')
[Timestamp('2019-11-14 16:37:01'), Timestamp('2019-11-14 16:37:02')]
[17, 16]
[17, 16]
25109 2019-11-14 16:37:01
25110 2019-11-14 16:37:01
25111 2019-11-14 16:37:01
25112 2019-11-14 16:37:01
25113 2019-11-14 16:37:01
25114 2019-11-14 16:37:01
25115 2019-11-14 16:37:01
25116 2019-11-14 16:37:01
25117 2019-11-14 16:37:01
25118 2019-11-14 16:37:01
25119 2019-11-14 16:37:01
25120 2019-11-14 16:37:01
25121 2019-11-14 16:37:01
25122 2019-11-14 16:37:01
25123 2019-11-14 16:37:01
25124 2019-11-14 16:37:01
25125 2019-11-14 16:37:01
25126 2019-11-14 16:37:02
25127 2019-11-14 16:37:02
25128 2019-11-14 16:37:02
25129 2019-11-14 16:37:02
25130 2019-11-14 16:37:02
25131 2019-11-14 16:37:02
25132 2019-11-14 16:37:02
25133 2019-11-14 16:37:02
25134 2019-11-14 16:37:02
25135 2019-11-14 16:37:0

41971 2019-11-14 17:05:26
41972 2019-11-14 17:05:26
41973 2019-11-14 17:05:26
41974 2019-11-14 17:05:26
41975 2019-11-14 17:05:26
41976 2019-11-14 17:05:26
41977 2019-11-14 17:05:26
41978 2019-11-14 17:05:26
41979 2019-11-14 17:05:26
41980 2019-11-14 17:05:26
41981 2019-11-14 17:05:26
41982 2019-11-14 17:05:26
41983 2019-11-14 17:05:26
41984 2019-11-14 17:05:26
41985 2019-11-14 17:05:26
41986 2019-11-14 17:05:26
41987 2019-11-14 17:05:26
41988 2019-11-14 17:05:27
41989 2019-11-14 17:05:27
41990 2019-11-14 17:05:27
41991 2019-11-14 17:05:27
41992 2019-11-14 17:05:27
41993 2019-11-14 17:05:27
41994 2019-11-14 17:05:27
41995 2019-11-14 17:05:27
41996 2019-11-14 17:05:27
41997 2019-11-14 17:05:27
41998 2019-11-14 17:05:27
41999 2019-11-14 17:05:27
42000 2019-11-14 17:05:27
42001 2019-11-14 17:05:27
42002 2019-11-14 17:05:27
42003 2019-11-14 17:05:27
42004 2019-11-14 17:05:27
42005 2019-11-14 17:05:27
42006 2019-11-14 17:05:27
42007 2019-11-14 17:05:28
42008 2019-11-14 17:05:28
42009 2019-1

57461 2019-11-14 17:31:30
57462 2019-11-14 17:31:30
57463 2019-11-14 17:31:30
57464 2019-11-14 17:31:30
57465 2019-11-14 17:31:30
57466 2019-11-14 17:31:30
57467 2019-11-14 17:31:30
57468 2019-11-14 17:31:30
57469 2019-11-14 17:31:30
57470 2019-11-14 17:31:30
57471 2019-11-14 17:31:30
57472 2019-11-14 17:31:30
57473 2019-11-14 17:31:30
57474 2019-11-14 17:31:30
[Timestamp('2019-11-14 18:19:37'), Timestamp('2019-11-14 18:19:38')]
[17, 16]
[17, 16]
85687 2019-11-14 18:19:37
85688 2019-11-14 18:19:37
85689 2019-11-14 18:19:37
85690 2019-11-14 18:19:37
85691 2019-11-14 18:19:37
85692 2019-11-14 18:19:37
85693 2019-11-14 18:19:37
85694 2019-11-14 18:19:37
85695 2019-11-14 18:19:37
85696 2019-11-14 18:19:37
85697 2019-11-14 18:19:37
85698 2019-11-14 18:19:37
85699 2019-11-14 18:19:37
85700 2019-11-14 18:19:37
85701 2019-11-14 18:19:37
85702 2019-11-14 18:19:37
85703 2019-11-14 18:19:37
85704 2019-11-14 18:19:38
85705 2019-11-14 18:19:38
85706 2019-11-14 18:19:38
85707 2019-11-14 18:19:38
857

 60%|████████████████████████████████████████████                             | 74988/124322 [00:43<00:43, 1140.03it/s]

In [23]:
from datetime import datetime

ts = int("1603729945")

# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

ts = int("1603729564")

# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

ts = int("1519603200")

# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))


2020-10-26 16:32:25
2020-10-26 16:26:04
2018-02-26 00:00:00


In [4]:
print(re.findall(r'%s(\d+)' %'_', '200215_00_00_00_329_OMAP_ATO.txt')[0])
if re.findall(r'%s(\d+)' %'_', '200215_00_00_00_329_OMAP_ATO.txt')[0] == '00':
    print('True')

00
True


In [7]:
# input_folder_path =  './EVR_T23_Car46_261020/'
# #input_folder_path = './Train 23 14NOV2019 Work Folder/raw logs'
# start_time = '2020/10/26 15:50:00' 
# end_time = '2020/10/26 16:50:00'
# #start_time = '2020/10/26 04:36:45'
# #end_time = '2020/10/26 04:37:09'
# sample_output_filename = ['./EVR_T23_Car46_261020/preprocessing_output/EVR_20201026_1550_to_20201026_1650.csv']


input_folder_path = './Train 23 14NOV2019 Work Folder/raw logs'
#input_folder_path = './Train 23 14NOV2019 Work Folder/raw logs'
start_time = '2019/11/14 17:00:00' 
end_time = '2019/11/14 18:00:00'
sample_output_filename = ['./Train 23 14NOV2019 Work Folder/preprocessing_output/EVR_20191114_1700_to_20191114_1800.csv']

def process_evr (input_folder_path, start_time, end_time):
    # Determine time difference between start and end time
    datetime_format = '%Y/%m/%d %H:%M:%S'
    start_time = datetime.datetime.strptime(start_time, datetime_format)
    end_time = datetime.datetime.strptime(end_time, datetime_format)
    start_time_new = start_time.strftime('%m/%d/%Y %H:%M:%S')
    end_time_new = end_time.strftime('%m/%d/%Y %H:%M:%S')
    # Check if EVR folder exist
    if not os.path.exists(os.path.join(input_folder_path, 'EVR')):
        print("EVR folder does not exist. Kindly check the folder strucutre. No EVR processing will be performed")
    else:
        print("EVR folder exist!")
        # Check if C2 file exist
        c2_pattern = 'EVR_Car\d+_\d+_C2.txt'
        norm_pattern = 'EVR_Car\d+_\d+.txt'
        c2_flag = False
        norm_flag = False
        for evr_file in os.listdir(os.path.join(input_folder_path, 'EVR')):
            if re.search(c2_pattern, evr_file):
                c2_filename = evr_file
                c2_flag = True
            if re.search(norm_pattern,evr_file):
                norm_filename = evr_file
                norm_flag = True

        if not (c2_flag and norm_flag):
            sys.exit("Missing EVR files! Kindly check EVR folder!")
        else:
            # Process C2 file
            df_c2 = pd.read_csv(os.path.join(input_folder_path, 'EVR', c2_filename), sep=";")
            df_c2['Date'] = pd.to_datetime(df_c2['Date'], format='%m/%d/%Y %H:%M:%S')
            df_c2_period = df_c2.loc[(df_c2['Date'] >= start_time_new) & (df_c2['Date'] <= end_time_new)]
            #df_c2_clean_timestamp = preprocess_timestamp_old(df_c2_period)
            df_c2_clean = process_dataframe(df_c2_period, 'C2')
            #df_c2_clean.to_csv('./result_c2.csv', index=False, header=True)
            
            # Processs Operating Data file
            df_od = pd.read_csv(os.path.join(input_folder_path, 'EVR', norm_filename), sep=";")
            df_od['Date'] = pd.to_datetime(df_od['Date'], format='%m/%d/%Y %H:%M:%S')
            df_od_period = df_od.loc[(df_od['Date'] >= start_time_new) & (df_od['Date'] <= end_time_new)]
            #df_od_clean_timestamp = preprocess_timestamp_old(df_od_period)
            df_od_clean = process_dataframe(df_od_period, 'OD')
            #df_od_clean.to_csv('./result_od.csv', index=False, header=True)
            
            # Merge other Operating Data to C2 Dataframe based on Timestampms (ordered manner)
            df_result = pd.merge_ordered(df_c2_clean, df_od_clean, how='outer', on='Timestampms')
            #df_result.to_csv('./result_4.csv', index=False, header=True)
            # Merge C2 and OD files first
            
            # Clean Timestamp
            df_result = preprocess_timestamp(df_result)
            
            
    return df_c2_clean, df_od_clean, df_result


df_c2_clean, df_od_clean, df_result = process_evr(input_folder_path, start_time, end_time)

output(sample_output_filename[0], df_result)
#unit_test(sample_output_filename[0], 'C2', df_c2_clean)  
#unit_test(sample_output_filename[0], 'OD', df_od_clean)  
unit_test_all(sample_output_filename[0], df_result)

EVR folder exist!
---Handling Duplicates, Split duplicate evenly---
13952 2019-11-14 17:00:01 1573750801000
13953 2019-11-14 17:00:01 1573750801333
13954 2019-11-14 17:00:01 1573750801666
---Handling Duplicates, Split duplicate evenly---
13961 2019-11-14 17:00:08 1573750808000
13962 2019-11-14 17:00:08 1573750808500
---Handling Duplicates, Split duplicate evenly---
13963 2019-11-14 17:00:09 1573750809000
13964 2019-11-14 17:00:09 1573750809333
13965 2019-11-14 17:00:09 1573750809666
---Handling Duplicates, Split duplicate evenly---
13966 2019-11-14 17:00:10 1573750810000
13967 2019-11-14 17:00:10 1573750810500
---Handling Duplicates, Split duplicate evenly---
13969 2019-11-14 17:00:11 1573750811000
13968 2019-11-14 17:00:11 1573750811500
---Handling Duplicates, Split duplicate evenly---
13975 2019-11-14 17:00:17 1573750817000
13976 2019-11-14 17:00:17 1573750817500
---Handling Duplicates, Split duplicate evenly---
13977 2019-11-14 17:00:18 1573750818000
13978 2019-11-14 17:00:18 157375

---Handling Duplicates, Split duplicate evenly---
14294 2019-11-14 17:04:21 1573751061000
14295 2019-11-14 17:04:21 1573751061500
---Handling Duplicates, Split duplicate evenly---
14296 2019-11-14 17:04:22 1573751062000
14297 2019-11-14 17:04:22 1573751062500
---Handling Duplicates, Split duplicate evenly---
14298 2019-11-14 17:04:23 1573751063000
14299 2019-11-14 17:04:23 1573751063500
---Handling Duplicates, Split duplicate evenly---
14301 2019-11-14 17:04:24 1573751064000
14300 2019-11-14 17:04:24 1573751064500
---Handling Duplicates, Split duplicate evenly---
14307 2019-11-14 17:04:30 1573751070000
14308 2019-11-14 17:04:30 1573751070500
---Handling Duplicates, Split duplicate evenly---
14309 2019-11-14 17:04:31 1573751071000
14310 2019-11-14 17:04:31 1573751071500
---Handling Duplicates, Split duplicate evenly---
14313 2019-11-14 17:04:34 1573751074000
14314 2019-11-14 17:04:34 1573751074333
14315 2019-11-14 17:04:34 1573751074666
---Handling Duplicates, Split duplicate evenly---


---Handling Duplicates, Split duplicate evenly---
15152 2019-11-14 17:17:14 1573751834000
15153 2019-11-14 17:17:14 1573751834500
---Handling Duplicates, Split duplicate evenly---
15171 2019-11-14 17:17:32 1573751852000
15172 2019-11-14 17:17:32 1573751852500
---Handling Duplicates, Split duplicate evenly---
15173 2019-11-14 17:17:33 1573751853000
15174 2019-11-14 17:17:33 1573751853500
---Handling Duplicates, Split duplicate evenly---
15175 2019-11-14 17:17:34 1573751854000
15176 2019-11-14 17:17:34 1573751854500
---Handling Duplicates, Split duplicate evenly---
15178 2019-11-14 17:17:35 1573751855000
15177 2019-11-14 17:17:35 1573751855500
---Handling Duplicates, Split duplicate evenly---
15179 2019-11-14 17:17:36 1573751856000
15180 2019-11-14 17:17:36 1573751856500
---Handling Duplicates, Split duplicate evenly---
15182 2019-11-14 17:17:38 1573751858000
15183 2019-11-14 17:17:38 1573751858333
15184 2019-11-14 17:17:38 1573751858666
---Handling Duplicates, Split duplicate evenly---


---Handling Duplicates, Split duplicate evenly---
15579 2019-11-14 17:22:37 1573752157000
15580 2019-11-14 17:22:37 1573752157500
---Handling Duplicates, Split duplicate evenly---
15581 2019-11-14 17:22:38 1573752158000
15582 2019-11-14 17:22:38 1573752158500
---Handling Duplicates, Split duplicate evenly---
15598 2019-11-14 17:22:53 1573752173000
15597 2019-11-14 17:22:53 1573752173500
---Handling Duplicates, Split duplicate evenly---
15607 2019-11-14 17:23:01 1573752181000
15606 2019-11-14 17:23:01 1573752181500
---Handling Duplicates, Split duplicate evenly---
15608 2019-11-14 17:23:02 1573752182000
15609 2019-11-14 17:23:02 1573752182500
---Handling Duplicates, Split duplicate evenly---
15610 2019-11-14 17:23:03 1573752183000
15611 2019-11-14 17:23:03 1573752183333
15612 2019-11-14 17:23:03 1573752183666
---Handling Duplicates, Split duplicate evenly---
15615 2019-11-14 17:23:04 1573752184000
15616 2019-11-14 17:23:04 1573752184250
15613 2019-11-14 17:23:04 1573752184500
15614 2019

---Handling Duplicates, Split duplicate evenly---
15909 2019-11-14 17:26:25 1573752385000
15910 2019-11-14 17:26:25 1573752385333
15908 2019-11-14 17:26:25 1573752385666
---Handling Duplicates, Split duplicate evenly---
15911 2019-11-14 17:26:26 1573752386000
15912 2019-11-14 17:26:26 1573752386500
---Handling Duplicates, Split duplicate evenly---
15914 2019-11-14 17:26:28 1573752388000
15915 2019-11-14 17:26:28 1573752388333
15916 2019-11-14 17:26:28 1573752388666
---Handling Duplicates, Split duplicate evenly---
15919 2019-11-14 17:26:29 1573752389000
15918 2019-11-14 17:26:29 1573752389333
15917 2019-11-14 17:26:29 1573752389666
---Handling Duplicates, Split duplicate evenly---
15920 2019-11-14 17:26:30 1573752390000
15921 2019-11-14 17:26:30 1573752390500
---Handling Duplicates, Split duplicate evenly---
15922 2019-11-14 17:26:31 1573752391000
15923 2019-11-14 17:26:31 1573752391333
15924 2019-11-14 17:26:31 1573752391666
---Handling Duplicates, Split duplicate evenly---
15932 2019

...

---Handling Duplicates, Split duplicate evenly---
18223 2019-11-14 17:53:24 1573754004000
18224 2019-11-14 17:53:24 1573754004333
18222 2019-11-14 17:53:24 1573754004666
---Handling Duplicates, Split duplicate evenly---
18225 2019-11-14 17:53:25 1573754005000
18226 2019-11-14 17:53:25 1573754005500
---Handling Duplicates, Split duplicate evenly---
18227 2019-11-14 17:53:26 1573754006000
18228 2019-11-14 17:53:26 1573754006500
---Handling Duplicates, Split duplicate evenly---
18229 2019-11-14 17:53:27 1573754007000
18230 2019-11-14 17:53:27 1573754007333
18231 2019-11-14 17:53:27 1573754007666
---Handling Duplicates, Split duplicate evenly---
18234 2019-11-14 17:53:29 1573754009000
18233 2019-11-14 17:53:29 1573754009500
---Handling Duplicates, Split duplicate evenly---
18236 2019-11-14 17:53:31 1573754011000
18237 2019-11-14 17:53:31 1573754011333
18238 2019-11-14 17:53:31 1573754011666
---Handling Duplicates, Split duplicate evenly---
18239 2019-11-14 17:53:32 1573754012000
18240 2019

---Handling Duplicates, Split duplicate evenly---
18590 2019-11-14 17:57:46 1573754266000
18589 2019-11-14 17:57:46 1573754266500
---Handling Duplicates, Split duplicate evenly---
18610 2019-11-14 17:58:03 1573754283000
18609 2019-11-14 17:58:03 1573754283200
18611 2019-11-14 17:58:03 1573754283400
18607 2019-11-14 17:58:03 1573754283600
18608 2019-11-14 17:58:03 1573754283800
---Handling Duplicates, Split duplicate evenly---
18612 2019-11-14 17:58:04 1573754284000
18613 2019-11-14 17:58:04 1573754284500
---Handling Duplicates, Split duplicate evenly---
18614 2019-11-14 17:58:05 1573754285000
18615 2019-11-14 17:58:05 1573754285333
18616 2019-11-14 17:58:05 1573754285666
---Handling Duplicates, Split duplicate evenly---
18623 2019-11-14 17:58:12 1573754292000
18624 2019-11-14 17:58:12 1573754292500
---Handling Duplicates, Split duplicate evenly---
18625 2019-11-14 17:58:13 1573754293000
18626 2019-11-14 17:58:13 1573754293500
---Handling Duplicates, Split duplicate evenly---
18627 2019

---Handling Duplicates, Fill gap with duplicate, Forward---
5616 2019-11-14 17:55:20 1573754120000
5617 2019-11-14 17:55:20 1573754121000
---Handling Duplicates, Fill gap with duplicate, Forward---
5647 2019-11-14 17:55:51 1573754151000
5648 2019-11-14 17:55:51 1573754152000
---Handling Duplicates, Fill gap with duplicate, Forward---
5649 2019-11-14 17:55:53 1573754153000
5650 2019-11-14 17:55:53 1573754154000
---Handling Duplicates, Fill gap with duplicate, Forward---
5651 2019-11-14 17:55:55 1573754155000
5652 2019-11-14 17:55:55 1573754156000
---Handling Duplicates, Fill gap with duplicate, Forward---
5653 2019-11-14 17:55:57 1573754157000
5654 2019-11-14 17:55:57 1573754158000
---Handling Duplicates, Fill gap with duplicate, Forward---
5659 2019-11-14 17:56:03 1573754163000
5660 2019-11-14 17:56:03 1573754164000
---Handling Duplicates, Fill gap with duplicate, Forward---
5666 2019-11-14 17:56:10 1573754170000
5667 2019-11-14 17:56:10 1573754171000
---Handling Duplicates, Split dupl

1000 1000
---Handling Duplicates, Split duplicate evenly---
18488.0 2019-11-14 17:56:30 1573754190000
18488.0 2019-11-14 17:56:30 1573754190500
1000 1000
---Handling Duplicates, Split duplicate evenly---
18531.0 2019-11-14 17:56:55 1573754215000
18531.0 2019-11-14 17:56:55 1573754215500
1000 1000
---Handling Duplicates, Split duplicate evenly---
18560.0 2019-11-14 17:57:24 1573754244000
18560.0 2019-11-14 17:57:24 1573754244500
1000 1000
---Handling Duplicates, Split duplicate evenly---
18597.0 2019-11-14 17:57:53 1573754273000
18597.0 2019-11-14 17:57:53 1573754273500
1000 1000
---Handling Duplicates, Split duplicate evenly---
18619.0 2019-11-14 17:58:08 1573754288000
18619.0 2019-11-14 17:58:08 1573754288500
1000 1000
---Handling Duplicates, Split duplicate evenly---
18635.0 2019-11-14 17:58:21 1573754301000
18635.0 2019-11-14 17:58:21 1573754301500
---Unit Test for entire Dataframe---
(4884, 1047) (4884, 1051)
Equality Between Sample Output and Self Processed:  False
Equality Betwee

AssertionError: 
Arrays are not equal

x and y nan location mismatch:
 x: array([13951., 13952., 13953., ..., 18821., 18822., 18823.])
 y: array([13951., 13952., 13953., ..., 18821., 18822., 18823.])

In [3]:
def process_dataframe(df, name):
    # Drop columns with NaNs
    df = df.dropna(how='any',axis=1)
            
    # Sort df based on column 'Date'
    df = df.sort_values(by='Date',ascending=True).reset_index(drop=True)
    
    # Convert datetime to timestamp (Unix) in milisecond resolution
    df['Timestamp'] = df.Date.values.astype(np.int64) // 10 ** 9 *1000
    # Add column for Real_Time Timestampms
    df['Timestampms'] = df['Timestamp']
    
     # Find duplicates datetime in column 'Date'
    df_duplicate = df[df.duplicated('Timestampms',keep=False)]
    for index, unique_datetime in enumerate (df_duplicate['Timestampms'].unique()):
        timestamp_df = df_duplicate.loc[df_duplicate['Timestampms'] == unique_datetime]
        if index < len(df_duplicate['Timestampms'].unique())-1:
            timestamp_df_next = df_duplicate.loc[df_duplicate['Timestampms'] == df_duplicate['Timestampms'].unique()[index+1]]
        if index > 0:
             timestamp_df_previous = df_duplicate.loc[df_duplicate['Timestampms'] == df_duplicate['Timestampms'].unique()[index-1]]
        difference_previous, difference_next = 0 , 0
        if timestamp_df.index[0]-1 >=0 and timestamp_df.index[-1]+1 <= len(df)-1:
            difference_previous = unique_datetime - df.loc[timestamp_df.index[0]-1,'Timestampms']
            difference_next = df.loc[timestamp_df.index[-1]+1,'Timestampms'] - unique_datetime
        elif timestamp_df.index[-1] == len(df)-1:
            difference_previous = unique_datetime - df.loc[timestamp_df.index[0]-1,'Timestampms']
        elif timestamp_df.index[0] == 0:
            difference_next = df.loc[timestamp_df.index[-1]+1,'Timestampms'] - unique_datetime
            
        #print(timestamp_df.index)
        if df.loc[timestamp_df.index[0],'Record Number'] < df.loc[timestamp_df.index[-1],'Record Number']:
            if (difference_previous == 1000 and difference_next >= 2000):
                print("---Handling Duplicates, Fill gap with duplicate, Forward---")
                for counter in range (0, len(timestamp_df.index)):
                    df.loc[timestamp_df.index[counter],'Timestampms'] = df.loc[timestamp_df.index[counter],'Timestampms'] + counter*1000
                    print(df.loc[timestamp_df.index[counter],'Record Number'], df.loc[timestamp_df.index[counter],'Date'], df.loc[timestamp_df.index[counter],'Timestampms'])
            elif(difference_previous >= 2000 and difference_next == 1000):
                # Loop backwards
                print("---Handling Duplicates, Fill gap with duplicate, Backward---")
                count = 0
                for counter in range (len(timestamp_df.index), 0, -1):
                    df.loc[timestamp_df.index[counter-1],'Timestampms'] = df.loc[timestamp_df.index[counter-1],'Timestampms'] - count*1000
                    count +=1
                    print(df.loc[timestamp_df.index[counter-1],'Record Number'], df.loc[timestamp_df.index[counter-1],'Date'], df.loc[timestamp_df.index[counter-1],'Timestampms'])
            #Implement fill preceding gap first
            elif(difference_previous == difference_next) and (difference_previous>1000 and difference_next>1000):
                print("---Handling Duplicates, Fill gap with duplicate, Forward and Backward---")
                mid_point = len(timestamp_df.index) // 2
                left_count, right_count = 0, 0
                for counter in range (1, len(timestamp_df.index)):
                    if counter % 2 == 0:
                        right_count +=1
                        df.loc[timestamp_df.index[mid_point+right_count],'Timestampms'] = df.loc[timestamp_df.index[mid_point],'Timestampms'] + right_count*1000
                        print(df.loc[timestamp_df.index[mid_point+right_count],'Record Number'], df.loc[timestamp_df.index[mid_point+right_count],'Date'], df.loc[timestamp_df.index[mid_point+right_count],'Timestampms'])
                    else:
                        left_count +=1
                        df.loc[timestamp_df.index[mid_point-left_count],'Timestampms'] = df.loc[timestamp_df.index[mid_point],'Timestampms'] - left_count*1000
                        print(df.loc[timestamp_df.index[mid_point-left_count],'Record Number'], df.loc[timestamp_df.index[mid_point-left_count],'Date'], df.loc[timestamp_df.index[mid_point-left_count],'Timestampms'])
            elif not(difference_previous == 0 and difference_next == 0) and ((difference_previous == difference_next) or 
                                                                       (difference_previous <= 1000 and difference_next <= 1000) or
                                                                       (difference_previous == 1000 and difference_next == 0) or
                                                                       (difference_previous == 0 and difference_next == 1000)):
                print("---Handling Duplicates, Split duplicate evenly---")
                # For every Timestamp (1000 millisecond), find the interval
                interval_ms = round(1000 / len(timestamp_df.index))
                for counter in range (0, len(timestamp_df.index)):
                    df.loc[timestamp_df.index[counter],'Timestampms'] = df.loc[timestamp_df.index[counter],'Timestampms'] + counter*interval_ms
                    print(df.loc[timestamp_df.index[counter],'Record Number'], df.loc[timestamp_df.index[counter],'Date'], df.loc[timestamp_df.index[counter],'Timestampms'])            
        else:
            if not(difference_previous == 0 and difference_next == 0) and ((difference_previous == difference_next) or 
                                                                       (difference_previous <= 1000 and difference_next <= 1000) or
                                                                       (difference_previous == 1000 and difference_next == 0) or
                                                                       (difference_previous == 0 and difference_next == 1000)):
                print("---Handling Duplicates, Split duplicate evenly---")
                # For every Timestamp (1000 millisecond), find the interval
                interval_ms = round(1000 / len(timestamp_df.index))
                for counter in range (0, len(timestamp_df.index)):
                    df.loc[timestamp_df.index[counter],'Timestampms'] = df.loc[timestamp_df.index[counter],'Timestampms'] + counter*interval_ms
                    print(df.loc[timestamp_df.index[counter],'Record Number'], df.loc[timestamp_df.index[counter],'Date'], df.loc[timestamp_df.index[counter],'Timestampms'])
        
    # Add Prefix to Columns name (e.g. EC2/EOD_***) except Timestampms
    if name == 'C2':
        df.columns = df.columns.map(lambda x : 'EC2_'+x if x !='Timestampms' else x)
    elif name == 'OD':
        df.columns = df.columns.map(lambda x : 'EOD_'+x if x !='Timestampms' else x)
        
    # Shift Timestampms to first column
    df = df[ ['Timestampms'] + [ col for col in df.columns if col != 'Timestampms' ] ]
    
    return df

def preprocess_timestamp(df):
    # Find duplicates datetime in colum 'Date'
    df_duplicate = df[df.duplicated('Timestampms',keep=False)]
    for unique_datetime in df_duplicate['Timestampms'].unique():
        timestamp_df = df_duplicate.loc[df_duplicate['Timestampms'] == unique_datetime]
        difference_previous, difference_next = 0 , 0
        #print(unique_datetime)
        if timestamp_df.index[0]-1 >=0 and timestamp_df.index[-1]+1 <= len(df)-1:
            difference_previous = unique_datetime - df.loc[timestamp_df.index[0]-1,'Timestampms']
            difference_next = df.loc[timestamp_df.index[-1]+1,'Timestampms'] - unique_datetime
            #print(difference_previous, difference_next)
        
        elif timestamp_df.index[-1] == len(df)-1:
            difference_previous = unique_datetime - df.loc[timestamp_df.index[0]-1,'Timestampms']
        elif timestamp_df.index[0] == 0:
            difference_next = df.loc[timestamp_df.index[-1]+1,'Timestampms'] - unique_datetime
        # Check if both (index[0]-1) and (index[-1]+1) are only difference of 1000 millisecond apart respectively
        print(difference_previous, difference_next)
        if not(difference_previous == 0 and difference_next == 0) and ((difference_previous == difference_next) or 
                                                                       (difference_previous <= 1000 and difference_next <= 1000) or
                                                                       (difference_previous == 1000 and difference_next == 0) or
                                                                       (difference_previous == 0 and difference_next == 1000)):
            print("---Handling Duplicates, Split duplicate evenly---")
            #print(difference_previous, difference_next)
            # Handle duplicates - Split duplicate evenly
            # For every Timestamp (1000 millisecond), find the interval
            interval_ms = round(1000 / len(timestamp_df.index))
            for counter in range (0, len(timestamp_df.index)):
                df.loc[timestamp_df.index[counter],'Timestampms'] = df.loc[timestamp_df.index[counter],'Timestampms'] + counter*interval_ms
                print(df.loc[timestamp_df.index[counter],'EC2_Record Number'], df.loc[timestamp_df.index[counter],'EC2_Date'], df.loc[timestamp_df.index[counter],'Timestampms'])

#         elif difference_previous == 1000 and difference_next >= 2000:
#             print("---Handling Duplicates, Fill gap with duplicate---")
#             for counter in range (0, len(timestamp_df.index)):
#                 df.loc[timestamp_df.index[counter],'Timestampms'] = df.loc[timestamp_df.index[counter],'Timestampms'] + counter*1000
#                 print(df.loc[timestamp_df.index[counter],'EC2_Record Number'], df.loc[timestamp_df.index[counter],'EC2_Date'], df.loc[timestamp_df.index[counter],'Timestampms'])
    
    # Group resulting same real timestamp together
    df = df.groupby("Timestampms").last().reset_index()
    
    # Remove similar record numbers for both EC2 and EOD
    #df_duplicate = df[df.duplicated('EC2_Record Number',keep='first')]
    filter_col = [col for col in df if col.startswith('EC2_')]
    df.loc[df.duplicated('EC2_Record Number',keep='first'), filter_col] = np.NaN
    filter_col = [col for col in df if col.startswith('EOD_')]
    df.loc[df.duplicated('EOD_Record Number',keep='first'), filter_col] = np.NaN
    
    df.to_csv('./result_2.csv', index=False, header=True)
    #print(filter_col)
#     print(df.head())
#     print(df.loc[8, 'Timestampms'], df.loc[8, 'EC2_Record Number'], df.loc[8, 'EOD_Record Number'])
#     print(df.loc[9, 'Timestampms'],df.loc[9, 'EC2_Record Number'], df.loc[9, 'EOD_Record Number'])
#     print(df.loc[10, 'Timestampms'],df.loc[10, 'EC2_Record Number'], df.loc[10, 'EOD_Record Number'])
        
        
    return df

def unit_test(sample_output_filename, name, df_test):
    print("---Unit Test for " + name + " Dataframe---")
    # Unit Test for C2
    # Import Output File
    df_output = pd.read_csv(sample_output_filename)
    # Only retrieve respective columns
    if name == 'C2':
        df_output = df_output.drop(df_output[(df_output['EC2_002_Record_Number'].isnull())].index)
        #drop column with prefix EOD
        df_output = df_output.loc[:, ~df_output.columns.str.startswith('EOD')]
    elif name == 'OD':
        df_output = df_output.drop(df_output[(df_output['EOD_002_Record_Number'].isnull())].index)
        #drop column with prefix EOD
        df_output = df_output.loc[:, ~df_output.columns.str.startswith('EC2')]
    
    df_output['epoch'] = df_output.epoch.values.astype(np.float64)
    df_output = df_output.reset_index(drop=True)
    print(df_output.shape, df_test.shape)    
    #Output to CSV
    #df_output.to_csv('./df_output.csv', index=False, header=True)

    #Output to CSV
    #df_test.to_csv('./df_test.csv', index=False, header=True)
    #df_drop_test = pd.read_csv('./df_test.csv')
    df_drop_test = df_test
    df_test['result'] = np.where(df_test['Timestampms'] == df_output['epoch'], 'True', 'False')
    df_test.to_csv('./df_test.csv', index=False, header=True)
    #df_drop_test = df_drop_test.sort_values(by='ATO_Real_Timestampms',ascending=True).reset_index(drop=True)
    # Assert whether sample output and self processed are equal
    assert_equal = omap.nan_equal(df_drop_test['Timestampms'].values, df_output['epoch'].values)
    print("Equality Between Sample Output and Self Processed: ", assert_equal)
    assert_equal = omap.nan_equal(df_drop_test['E'+name+'_Record Number'].values, df_output['E'+name+'_002_Record_Number'].values)
    print("Equality Between Sample Output and Self Processed: ", assert_equal)
    
    #df_drop_test.columns = df_output.columns
    #print(np.testing.assert_allclose(df_drop_test.values, df_output.values, rtol=1e-10, atol=0))
    #print(pd.testing.assert_frame_equal(df_drop_test, df_output, check_dtype=False))
    #print(df_drop_test.compare(df_output, align_axis=0))
    #assert_equal = nan_equal(df_drop_test.values, df_output.values)
    #assert_equal = nan_equal(df_drop['ATO_* General'].values, df_output['ATO_0101__General'].values)
    #print("Equality Between Sample Output and Self Processed: ", assert_equal)
    #print(np.testing.assert_equal(df_drop_test.values, df_output.values))
    
def unit_test_all(sample_output_filename, df_test):
    print("---Unit Test for entire Dataframe---")
    # Import Output File
    df_output = pd.read_csv(sample_output_filename)
    print(df_output.shape, df_test.shape)
    assert_equal = omap.nan_equal(df_test['Timestampms'].values, df_output['epoch'].values)
    print("Equality Between Sample Output and Self Processed: ", assert_equal)
    df_test['result'] = np.where(df_test['Timestampms'] == df_output['epoch'], 'True', 'False')
    #df_test.to_csv('./df_test.csv', index=False, header=True)
        
    df_test['result_1'] = np.where(df_test['EC2_Record Number'] == df_output['EC2_002_Record_Number'], 'True', 'False')
    df_test.to_csv('./df_test.csv', index=False, header=True)
    assert_equal = omap.nan_equal(df_test['EC2_Record Number'].values, df_output['EC2_002_Record_Number'].values)
    print("Equality Between Sample Output and Self Processed: ", assert_equal)
    assert_equal = omap.nan_equal(df_test['EOD_Record Number'].values, df_output['EOD_002_Record_Number'].values)
    print("Equality Between Sample Output and Self Processed: ", assert_equal)
    print(np.testing.assert_equal(df_test['EC2_Record Number'].values, df_output['EC2_002_Record_Number'].values))

In [6]:
def preprocess_timestamp_old(df):
    # Drop columns with NaNs
    df = df.dropna(how='any',axis=1)
            
    # Sort df based on column 'Date'
    df = df.sort_values(by='Date',ascending=True).reset_index(drop=True)

    # Convert datetime to timestamp (Unix) in milisecond resolution
    df['Timestamp'] = df.Date.values.astype(np.int64) // 10 ** 9 *1000
    # Add column for Real_Time Timestampms
    df['Timestampms'] = df['Timestamp']

    # Find duplicates datetime in colum 'Date'
    df_duplicate = df[df.duplicated('Timestamp',keep=False)]
    for unique_datetime in df_duplicate['Timestamp'].unique():
        timestamp_df = df_duplicate.loc[df_duplicate['Timestamp'] == unique_datetime]
        difference_previous, difference_next = 0 , 0
        #print(unique_datetime)
        if timestamp_df.index[0]-1 >=0 and timestamp_df.index[-1]+1 <= len(df)-1:
            difference_previous = unique_datetime - df.loc[timestamp_df.index[0]-1,'Timestamp']
            difference_next = df.loc[timestamp_df.index[-1]+1,'Timestamp'] - unique_datetime
            #print(difference_previous, difference_next)
        
        elif timestamp_df.index[-1] == len(df)-1:
            difference_previous = unique_datetime - df.loc[timestamp_df.index[0]-1,'Timestamp']
        elif timestamp_df.index[0] == 0:
            difference_next = df.loc[timestamp_df.index[-1]+1,'Timestamp'] - unique_datetime
        # Check if both (index[0]-1) and (index[-1]+1) are only difference of 1000 millisecond apart respectively
        print(difference_previous, difference_next)
        if not(difference_previous == 0 and difference_next == 0) and ((difference_previous == difference_next) or 
                                                                       (difference_previous == 1000 and difference_next == 1000) or
                                                                       (difference_previous == 1000 and difference_next == 0) or
                                                                       (difference_previous == 0 and difference_next == 1000)):
            print("---Handling Duplicates, Split duplicate evenly---")
            #print(difference_previous, difference_next)
            # Handle duplicates - Split duplicate evenly
            # For every Timestamp (1000 millisecond), find the interval
            interval_ms = round(1000 / len(timestamp_df.index))
            for counter in range (0, len(timestamp_df.index)):
                df.loc[timestamp_df.index[counter],'Timestampms'] = df.loc[timestamp_df.index[counter],'Timestampms'] + counter*interval_ms
                print(df.loc[timestamp_df.index[counter],'Record Number'], df.loc[timestamp_df.index[counter],'Date'], df.loc[timestamp_df.index[counter],'Timestampms'])
        elif difference_previous == 1000 and difference_next >= 2000:
            print("---Handling Duplicates, Fill gap with duplicate---")
            for counter in range (0, len(timestamp_df.index)):
                df.loc[timestamp_df.index[counter],'Timestampms'] = df.loc[timestamp_df.index[counter],'Timestampms'] + counter*1000
                print(df.loc[timestamp_df.index[counter],'Record Number'], df.loc[timestamp_df.index[counter],'Date'], df.loc[timestamp_df.index[counter],'Timestampms'])
                
    # Add Prefix to Columns name (ATO_***) except Timestampms
    #df.columns = df.columns.map(lambda x : 'ATO_'+x if x !='Timestampms' and x!='Real_Timestampms' else x)
    # Shift Timestampms to first column
    df = df[ ['Timestampms'] + [ col for col in df.columns if col != 'Timestampms' ] ]
    return df

In [ ]:
  # Find duplicates datetime in column 'Date'
    df_duplicate = df[df.duplicated('Timestampms',keep=False)]
    for index in range (0, len(df_duplicate['Timestampms'].unique())-1):
        timestamp_df = df_duplicate.loc[df_duplicate['Timestampms'] == df_duplicate['Timestampms'].unique()[index]]
        #print(timestamp_df.index)
        timestamp_df_next = df_duplicate.loc[df_duplicate['Timestampms'] == df_duplicate['Timestampms'].unique()[index+1]]
        # Duplicates are side by side
        if(timestamp_df_next.index[0]-timestamp_df.index[-1]==1) and (df.loc[timestamp_df.index[0],'Record Number'] < df.loc[timestamp_df.index[-1],'Record Number']):
            print("---Handling Duplicates, Fill gap with duplicate---")
            for counter in range (0, len(timestamp_df.index)):
                df.loc[timestamp_df.index[counter],'Timestampms'] = df.loc[timestamp_df.index[counter],'Timestampms'] + counter*1000
                print(df.loc[timestamp_df.index[counter],'Record Number'], df.loc[timestamp_df.index[counter],'Date'], df.loc[timestamp_df.index[counter],'Timestampms'])

In [6]:
def output(sample_output_filename, df_result):
    # Import Output File
    df_output = pd.read_csv(sample_output_filename)
    
    df_combine = pd.DataFrame()
    df_combine = df_combine.assign(epoch = df_output['epoch'])
    df_combine = df_combine.assign(Timestampms = df_result['Timestampms']) 
    df_combine = df_combine.assign(EC2_Record_Number = df_result['EC2_Record Number']) 
    df_combine = df_combine.assign(EC2_Date = df_result['EC2_Date']) 
    df_combine = df_combine.assign(EOD_Record_Number = df_result['EOD_Record Number']) 
    df_combine = df_combine.assign(EOD_Date = df_result['EOD_Date']) 

    
    df_combine.to_csv('./result_combine_1.csv', index=False, header=True)